# Logistic regression

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 
import os
path = os.getcwd() + '/ex2data1.txt'
data = pd.read_csv(path, header=None, names=['Exam 1', 'Exam 2', 'Admitted'])

**Example 0:**

In [ ]:
# Right there I use different functions that I learned earlier, to get familiar with the data
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.dropna()

In [ ]:
data_new = (data - data.mean())/data.std()  # performing 'feature normalization' (I won't use normalized y!!!)
data_new

**Example 1:**

In [ ]:
data.insert(0, '1s', 1)  # with function 'insert' I added column of 1s at the beginning
data_new.insert(0, '1s', 1)

In [ ]:
X = data[['1s', 'Exam 1', 'Exam 2']]
y = data['Admitted']

In [ ]:
X = np.array(X.values).T  # changing to numpy
y = np.array(y.values, dtype=float)

**Example 2:**

In [ ]:
theta = np.zeros(3)

In [ ]:
# Plotting the data
admitted = []
not_admitted = []
for i in range(X.shape[1]):
    if y[i] == 1:
        admitted.append(X[1:, i])
    else:
        not_admitted.append(X[1:, i])
admitted = np.array(admitted)
not_admitted = np.array(not_admitted)

plt.figure(figsize=(20, 10))
'''fig, ax = plt.subplots()
fig.set_size_inches(20, 10)

plt.scatter(X.T[:, 1], X.T[:, 2], c=y)'''
plt.scatter(admitted[:, 0], admitted[:, 1], color='green', label='admitted')
plt.scatter(not_admitted[:, 0], not_admitted[:, 1], color='red', label='not admitted')
plt.xlabel('Iteration number')
plt.ylabel('Cost function value')
plt.legend(loc='upper right')

plt.show()

**Example 3:**

In [ ]:
# Implemented logistic function 
def sig(t):
    out = 1/(1 + np.exp(-t))
    return out

**Example 4:**

In [ ]:
# Plotting the results of written function
test = np.arange(-5, 5, 0.5)
result = sig(test)

plt.plot(test, result)
plt.xlabel('Iteration number')
plt.ylabel('Cost function value')
plt.show()

**Example 5:**

In [ ]:
# Implemented vectorized function 'cost'
def cost(theta_p, X_p, y_p):
    m = y_p.shape[0]
    h = sig(theta_p@X_p)
    y_0 = -(1 - y_p.T)@(np.log(1 - h)).T
    y_1 = -y_p.T@(np.log(h)).T
    c = (y_1 + y_0)/m
    return c

In [ ]:
theta = np.zeros(3)

cost(theta, X, y)  # quick chceck of the outcome

**Example 6:**

In [ ]:
# Implemented simple gradient function
def simple_gradient(X_p, y_p, theta_p, alpha_p, it_p):
    # it - iteration nb.
    m = y_p.shape[0]
    costs = 0
    for i in range(it_p):
        h = sig(theta_p@X_p)
        theta_p = theta_p - alpha_p*((h - y_p)@X_p.T)/m
        costs = cost(theta_p, X_p, y_p)
    return theta_p, costs

In [ ]:
# Preparing data to execute the function
alpha = 1
it = 150
theta = np.zeros(3)

X_new = data_new[['1s', 'Exam 1', 'Exam 2']]
X_new = np.array(X_new.values).T

theta_new, final_cost = simple_gradient(X_new, y, theta, alpha, it)

In [ ]:
theta_new  # new theta values, calculated by simple_gradient function

In [ ]:
final_cost  # final cost, calculated by simple_gradient function

**Example 7:**

In [ ]:
result = (sig(theta_new@X_new) >= 0.5).astype(int)  # applying threshold

In [ ]:
comparison = np.array([y[i] == result[i] for i in range(y.shape[0])])

In [ ]:
comparison  # True means, that the prediction was correct

In [ ]:
accuracy = len(comparison[comparison != False])/len(comparison)  # final accuracy of the algorithm
accuracy

**Example 8:**

In [ ]:
# calculating coefficients for regression
c = -theta_new[0]/theta_new[2]
m = -theta_new[1]/theta_new[2]

boundary = m*X_new.T[:, 1] + c

In [ ]:
# Plotting the data with regression 
admitted = []
not_admitted = []
for i in range(X.shape[1]):
    if y[i] == 1:
        admitted.append(X_new[1:, i])
    else:
        not_admitted.append(X_new[1:, i])
admitted = np.array(admitted)
not_admitted = np.array(not_admitted)

plt.figure(figsize=(20, 10))
#fig, ax = plt.subplots()
#fig.set_size_inches(20, 10)

plt.scatter(admitted[:, 0], admitted[:, 1], color='green', label='admitted')
plt.scatter(not_admitted[:, 0], not_admitted[:, 1], color='red', label='not admitted')
plt.plot(X_new.T[:, 1], boundary)
plt.xlabel('Iteration number')
plt.ylabel('Cost function value')
plt.legend(loc='upper right')


plt.show()

# Logistic regression - Python

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
 
iris = datasets.load_iris()
X = iris.data[:, :2]  # we are analyzing only 2 parameters
Y = iris.target

**Example 1, 2:**

In [ ]:
logreg = LogisticRegression(C=10.0)  # I use there default optimalization algorithm, with regularization coefficient 
                                    # equal to 10 (best outcome)                
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)  # split the data to train and test dataset

logreg.fit(X_train, Y_train)  # train the model
Y_predicted = logreg.predict(X_test)

In [ ]:
# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
h = .02  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = logreg.predict(np.c_[xx.ravel(), yy.ravel()])
 
# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(10, 8))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)
 
# Plot also the training points
plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k', cmap=plt.cm.Paired)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')
 
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())
 
plt.show()

**Example 3:**

In [ ]:
predict = logreg.predict_proba(X_test)  # used function to calculate probability of belonging

In [ ]:
# Bar plot used to compare probabilities
plt.figure(figsize=(20, 5))

X_len = X_test.shape[0]
X_name = np.linspace(1, 45, 45)
X_axis = np.arange(X_len)

plt.bar(X_axis - 0.3, predict[:, 0], width=0.3, label = 'Class 0')
plt.bar(X_axis, predict[:, 1], width=0.3, label = 'Class 1')
plt.bar(X_axis + 0.3, predict[:, 2], width=0.3, label = 'Class 2')
plt.xticks(X_axis, X_name)
plt.xlim(-1, 45)
plt.xlabel('Feature')
plt.ylabel('Probability of being a member of a given class')
plt.legend(loc='upper right')


plt.show()

**Example 4:**

In [ ]:
mean_squared_error(Y_test, Y_predicted)  # calculated MSE

In [ ]:
r2_score(Y_test, Y_predicted)  # calculated R2

In [ ]:
mean_absolute_error(Y_test, Y_predicted)  # calculated mean error rate